In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('pyspark_test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 02:27:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# Global variables

color = 'green'
year = '2020'
prefix = f'../data/raw/{color}/{year}'

In [4]:
# Create schema for spark df
file_name = f'{color}_tripdata_{year}_01.csv.gz'

import pandas as pd
df_pandas = pd.read_csv(f'{prefix}/01/{file_name}', nrows=1000)

print(prefix)

# spark.createDataFrame(df_pandas).schema

../csv_files/data/raw/yellow/2020


In [5]:
# modify schema

green_schema = types.StructType([
    types.StructField('VendorID', types.IntegerType(), True), 
    types.StructField('lpep_pickup_datetime', types.TimestampType(), True), 
    types.StructField('lpep_dropoff_datetime', types.TimestampType(), True), 
    types.StructField('store_and_fwd_flag', types.StringType(), True), 
    types.StructField('RatecodeID', types.IntegerType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('passenger_count', types.IntegerType(), True), 
    types.StructField('trip_distance', types.DoubleType(), True), 
    types.StructField('fare_amount', types.DoubleType(), True), 
    types.StructField('extra', types.DoubleType(), True), 
    types.StructField('mta_tax', types.DoubleType(), True), 
    types.StructField('tip_amount', types.DoubleType(), True), 
    types.StructField('tolls_amount', types.DoubleType(), True), 
    types.StructField('ehail_fee', types.DoubleType(), True), 
    types.StructField('improvement_surcharge', types.DoubleType(), True), 
    types.StructField('total_amount', types.DoubleType(), True), 
    types.StructField('payment_type', types.IntegerType(), True), 
    types.StructField('trip_type', types.IntegerType(), True), 
    types.StructField('congestion_surcharge', types.DoubleType(), True)
    ])

yellow_schema = types.StructType([
    types.StructField('VendorID', types.IntegerType(), True), 
    types.StructField('tpep_pickup_datetime', types.TimestampType(), True), 
    types.StructField('tpep_dropoff_datetime', types.TimestampType(), True), 
    types.StructField('passenger_count', types.IntegerType(), True), 
    types.StructField('trip_distance', types.DoubleType(), True), 
    types.StructField('RatecodeID', types.IntegerType(), True), 
    types.StructField('store_and_fwd_flag', types.StringType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('payment_type', types.IntegerType(), True), 
    types.StructField('fare_amount', types.DoubleType(), True), 
    types.StructField('extra', types.DoubleType(), True), 
    types.StructField('mta_tax', types.DoubleType(), True), 
    types.StructField('tip_amount', types.DoubleType(), True), 
    types.StructField('tolls_amount', types.DoubleType(), True), 
    types.StructField('improvement_surcharge', types.DoubleType(), True), 
    types.StructField('total_amount', types.DoubleType(), True), 
    types.StructField('congestion_surcharge', types.DoubleType(), True)
    ])

In [12]:
# read csv files with schema and upload partitioned parquet files

for month in range(1, 13):
    print(f"processing {color} data for {year}, {month}")
    input_path = f'{prefix}/{month:02d}/'
    output_path = f'../data/pq/{color}/{year}/{month:02d}/'

    df_spark = spark.read \
        .option("header", "true") \
        .schema(yellow_schema if color == 'yellow' else green_schema) \
        .csv(input_path)

    df_spark \
        .repartition(4) \
        .write.parquet(output_path)

processing green data for 2021, 1
processing green data for 2021, 2
processing green data for 2021, 3
processing green data for 2021, 4
processing green data for 2021, 5


processing green data for 2021, 6
processing green data for 2021, 7
processing green data for 2021, 8
processing green data for 2021, 9


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/ramazan/DE_zoomcamp_2024/module_5/csv_files/data/raw/green/2021/09.